In [1]:
import multiprocessing as mp
import itertools as it
import functools as ft
import pickle
import sys
import numpy as np
import pandas as pd
import time
import sklearn
import sklearn.preprocessing as pre
import scipy.sparse as sp

In [34]:
dat = pd.read_csv('/Users/isrobson/Downloads/cord_blood_kinases.csv', sep=',', header=0, index_col=0);

/Users/isrobson/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [35]:
dat

,barcode,symbol
1,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,STK4
2,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,RIOK3
3,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,PDK2
4,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,MAPK7
5,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,CDK10
6,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,SMG1
7,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,MAP3K9
8,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,PRKCH
9,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,DGKA
10,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,ATM


In [36]:
def get_cell_sets(row, oe_csr):
    return oe_csr[row['lower']:row['upper']].sum(axis=0)

def first_candidates(cells, cell_sets, min_shared_cells):
    count_filter = cell_sets.apply(len) > min_shared_cells
    return list(map(lambda x: frozenset([x]), ((cells[count_filter])))), {frozenset([x]):y for x,y in cell_sets[count_filter].to_dict().items()}

def intersector(tuple_of_candidates, tuple_of_sets):
    return ft.reduce(lambda x,y: x.union(y), tuple_of_candidates), tuple_of_sets[0] & tuple_of_sets[1]

def cell_set_getter(input_list, cell_sets):
    for i in input_list:
        yield cell_sets[i]

def pickle_cells(cells, cell_sets, k):
    '''These files are gonna be decently big. Do not want to keep them in memory.'''
    with open('cell_' + str(k) + '.pickle', 'wb') as f:
        pickle.dump(cells, f, pickle.HIGHEST_PROTOCOL)
    with open('cell_sets_' + str(k) + '.pickle', 'wb') as f:
        pickle.dump(cell_sets, f, pickle.HIGHEST_PROTOCOL)

In [37]:
from sklearn.preprocessing import LabelBinarizer as LB

dat['barcode'].unique()


array(['MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1',
       'MantonCB1_HiSeq_1-AAACCTGCACAGATTC-1',
       'MantonCB1_HiSeq_1-AAACCTGCACCCAGTG-1', ...,
       'MantonCB8_HiSeq_8-TTTGTCAGTCCCTTGT-1',
       'MantonCB8_HiSeq_8-TTTGTCAGTCGTTGTA-1',
       'MantonCB8_HiSeq_8-TTTGTCATCCAGATCA-1'], dtype=object)

In [38]:
total_cells = dat['barcode'].nunique()

min_shared_cells = 100
min_percent_cells = None
if(min_percent_cells is not None):
    min_shared_cells = int(min_percent_cells * total_cells)

cell_id_dict = {y:x for x,y in enumerate(dat['symbol'].unique())}
dat['symbol'] = dat['symbol'].map(cell_id_dict)
cells = dat['symbol'].unique()

barcode_id_dict = {y:x for x,y in enumerate(dat['barcode'].unique())}
dat['barcode'] = dat['barcode'].map(barcode_id_dict)

lb = LB()
lb.fit(dat['barcode'].unique().tolist())

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [39]:
dat = dat.iloc[:10000,:]

In [40]:
bar = lambda x: lb.transform([x])

dat['barcode'] = dat['barcode'].map(bar)

cell_sets = dat.groupby('symbol')['barcode'].apply(ax_0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
temp = pd.DataFrame(np.vstack([i for i in cell_sets])[:,:600])

In [48]:
temp.columns = list(barcode_id_dict.keys())[:600]

In [61]:
temp['symbol'] = list(cell_id_dict.keys())[:temp.shape[0]]

In [62]:
temp.set_index('symbol')

,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,MantonCB1_HiSeq_1-AAACCTGCACAGATTC-1,MantonCB1_HiSeq_1-AAACCTGCACCCAGTG-1,MantonCB1_HiSeq_1-AAACCTGCACTGTGTA-1,MantonCB1_HiSeq_1-AAACCTGGTTCCACGG-1,MantonCB1_HiSeq_1-AAACCTGTCCTGCCAT-1,MantonCB1_HiSeq_1-AAACCTGTCTCCCTGA-1,MantonCB1_HiSeq_1-AAACGGGCAATGGAGC-1,MantonCB1_HiSeq_1-AAACGGGGTAAGGATT-1,MantonCB1_HiSeq_1-AAACGGGGTCGAGTTT-1,...,MantonCB1_HiSeq_1-ATGGGAGTCTCTGCTG-1,MantonCB1_HiSeq_1-ATGTGTGCATGGTCAT-1,MantonCB1_HiSeq_1-ATGTGTGGTGACAAAT-1,MantonCB1_HiSeq_1-ATGTGTGTCGCATGGC-1,MantonCB1_HiSeq_1-ATGTGTGTCTAACCGA-1,MantonCB1_HiSeq_1-ATGTGTGTCTATCCCG-1,MantonCB1_HiSeq_1-ATTACTCAGATGAGAG-1,MantonCB1_HiSeq_1-ATTACTCAGCAGGCTA-1,MantonCB1_HiSeq_1-ATTACTCCACTTGGAT-1,MantonCB1_HiSeq_1-ATTACTCCAGCCAATT-1
symbol,,,,,,,,,,,,,,,,,,,,,
STK4,1,1,1,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
RIOK3,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
PDK2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MAPK7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDK10,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SMG1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MAP3K9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PRKCH,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
DGKA,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
temp.to_csv('first_10000_heatmap.csv')

In [17]:
def fast_gather_gene_sets(dat, min_shared_cells = 100, min_percent_cells = None, max_cluster_size = sys.maxsize):
    st = time.time()
    begin = st
    cores = max(mp.cpu_count()-1, 1)
    
    total_cells = dat['barcode'].nunique()
    
    if(min_percent_cells is not None):
        min_shared_cells = int(min_percent_cells * total_cells)

    cell_id_dict = {y:x for x,y in enumerate(dat['symbol'].unique())}
    dat['symbol'] = dat['symbol'].map(cell_id_dict)
    cells = dat['symbol'].unique()
    
    barcode_id_dict = {y:x for x,y in enumerate(dat['barcode'].unique())}
    dat['barcode'] = dat['barcode'].map(barcode_id_dict)
    
    cell_sets = dat.groupby('symbol')['barcode'].apply(set)
    
    en = time.time()
    
    print('Formatted data in ' + str(en-st) + ' seconds')
    
    cells, cell_sets = first_candidates(cells, cell_sets, min_shared_cells)
    
    print(str(len(cells)) + ' genes made have > ' + str(min_shared_cells) + ' cells')
    
    k = 2
    n = len(cells)
    
    pickle_cells(cells, cell_sets, k)
    
    while(len(cells) > 0 and k < max_cluster_size):
        st = time.time()
        
        candidates_left, candidates_right = zip(*list(filter(lambda x: len(x[0]|x[1]) == k, it.combinations(cells, 2))))
        left_gen = cell_set_getter(candidates_left, cell_sets)
        right_gen = cell_set_getter(candidates_right, cell_sets)        
        gener = ((x, y) for x, y in map(intersector, *(zip(candidates_left, candidates_right),zip(left_gen, right_gen))) if len(y)>min_shared_cells)

        try:
            cells, cell_sets = zip(*gener)
            cell_sets = dict(zip(cells, cell_sets))
            cells = list(cells)
        except:
            print('Ran out of candidates! Last size: ' + str(k))
            print('Total time: ' + str(en - begin) + ' seconds')
        
        k+= 1
        n = len(cells)
        
        en = time.time()
        
        print('Found ' + str(n) + ' remaining gene clusters with > ' + str(min_shared_cells) + ' of size: ' +str(k-1))
        print('Iteration took: ' + str(en-st) + ' seconds')
        
        if(n == 0):
            print('Terminated! Total run time: ' + str(en - begin) + ' seconds')
        else:
            print('Pickling!')
            pickle_cells(cells, cell_sets, k-1)
        

In [ ]:
fast_gather_gene_sets(dat, min_percent_cells = 0.01)

Formatted data in 3.3792808055877686 seconds
303 genes made have > 2738 cells
Found 5182 remaining gene clusters with > 2738 of size: 2
Iteration took: 18.198082208633423 seconds
Pickling!
Found 27906 remaining gene clusters with > 2738 of size: 3
Iteration took: 131.35212016105652 seconds
Pickling!
Ran out of candidates! Last size: 4
Total time: 154.87209296226501 seconds
Found 27906 remaining gene clusters with > 2738 of size: 4
Iteration took: 2965.364686727524 seconds
Pickling!
